### Import Packages

In [24]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import IterativeImputer
from imblearn.over_sampling import SMOTE

### Import Data

In [25]:
application = pd.read_csv("original_data/application_record.csv")
credit = pd.read_csv("original_data/credit_record.csv")

application_cat_cols = application.select_dtypes(include=["object"]).columns
application_num_cols = application.select_dtypes(exclude=["object"]).columns
credit_cat_cols = credit.select_dtypes(include=["object"]).columns
credit_num_cols = credit.select_dtypes(exclude=["object"]).columns

In [26]:
application.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


### Encoding Categorical Data

In [27]:
# encoder = LabelEncoder()
encoders = {col: LabelEncoder() for col in application_cat_cols}

application[application_cat_cols] = application[application_cat_cols].apply(lambda series: pd.Series(
    encoders[series.name].fit_transform(series[series.notnull()]),
    index=series[series.notnull()].index
))

# showing a glimpse of the the new values in categorial columns
application.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,1,1,1,0,112500.0,4,4,1,1,-21474,-1134,1,0,0,0,16.0,2.0
3,5008808,0,0,1,0,270000.0,0,4,3,1,-19110,-3051,1,0,1,1,14.0,1.0
4,5008809,0,0,1,0,270000.0,0,4,3,1,-19110,-3051,1,0,1,1,14.0,1.0


In [28]:
with open("objects/encoders.obj", "wb") as obj_handler:
    pickle.dump(encoders, obj_handler)

In [29]:
le = encoders["NAME_HOUSING_TYPE"]
dict(zip(le.classes_, le.transform(le.classes_)))

{'Co-op apartment': 0,
 'House / apartment': 1,
 'Municipal apartment': 2,
 'Office apartment': 3,
 'Rented apartment': 4,
 'With parents': 5}

### Imputation

There are over 44% of data in the column OCCUPATION_TYPE is missing.

We can use an Iterative Imputer to fill in the missing values:

In [30]:
application

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,1,1,1,0,112500.0,4,4,1,1,-21474,-1134,1,0,0,0,16.0,2.0
3,5008808,0,0,1,0,270000.0,0,4,3,1,-19110,-3051,1,0,1,1,14.0,1.0
4,5008809,0,0,1,0,270000.0,0,4,3,1,-19110,-3051,1,0,1,1,14.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,1,0,1,0,135000.0,1,4,2,1,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,0,0,0,0,103500.0,4,4,3,1,-15939,-3007,1,0,0,0,8.0,1.0
438554,6841878,0,0,0,0,54000.0,0,1,3,5,-8169,-372,1,1,0,0,14.0,1.0
438555,6842765,0,0,1,0,72000.0,1,4,1,1,-21673,365243,1,0,0,0,NaN,2.0


In [31]:
imputer = IterativeImputer()
scaler = MinMaxScaler()
application = pd.DataFrame(scaler.fit_transform(application), columns=application.columns)
application = pd.DataFrame(imputer.fit_transform(application), columns=application.columns)
application = pd.DataFrame(scaler.inverse_transform(application), columns=application.columns)

print(f"Any null value in application record: {application.isnull().values.any()}")

Any null value in application record: False


Because the column OCCUPATION TYPE now contains non-integer values, we need to transform it again:

In [32]:
application["OCCUPATION_TYPE"] = application["OCCUPATION_TYPE"].apply(lambda x: np.round(x))
application["OCCUPATION_TYPE"].value_counts()

8.0     109209
46.0     59512
3.0      43007
14.0     41098
10.0     35488
4.0      26090
6.0      20864
7.0      20470
0.0      15985
11.0     13520
45.0     11743
2.0       8076
16.0      7993
9.0       6515
1.0       5845
12.0      3456
44.0      2235
15.0      2044
47.0      1839
17.0      1665
13.0      1041
5.0        862
Name: OCCUPATION_TYPE, dtype: int64

### Data Labelling

From vintage analysis, we can categorise a customer as "bad" if the customer defaults (90 days or more overdue) in the credit record

The STATUS column in the credit record is defined as follows:

| STATUS | Meaning |
|--------|---------|
| 0      | 1-29 days overdue |
| 1      | 30-59 days overdue|
| 2      | 60-89 days overdue|
| 3      | 90-119 days overdue|
|4 | 120-149 days overdue|
| 5| Overdue or bad debts, write-offs for more than 150 days|
|C| Loan paid off |
|X| No loan|

Therefore, we can define a NEW_STATUS column as follows:

| NEW_STATUS | New meaning | STATUS |
|--------|-------------|----|
| 0      | No loan overdue for more than 90 days | {"C", "X", "0", "1", "2"} |
| 1      | Loan overdue for more than 90 days | {"3", "4", "5"}|

In [33]:
credit["NEW_STATUS"] = credit["STATUS"].apply(lambda s: 1 if s in ["3", "4", "5"] else 0)
credit["NEW_STATUS"].value_counts(normalize=True)

0    0.997868
1    0.002132
Name: NEW_STATUS, dtype: float64

Furthermore, we define the current status of a customer as "bad" if there is any history of him/her defaulting from credit record. 

In view of this, the credit record is aggregated to get the maximum value of STATUS for each customer ID. 

In [34]:
credit_agg = credit.groupby("ID").agg(max).reset_index()
credit_agg["NEW_STATUS"].value_counts(normalize=True)

0    0.992802
1    0.007198
Name: NEW_STATUS, dtype: float64

The above results showed clear signs of imbalance dataset. Extra actions will be needed in later section before training the model.

### Data Merging

In [35]:
df = application.join(credit_agg.set_index("ID"), on="ID", how="inner")
df.head()

/Users/elvistkf/miniforge3/envs/DS/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1205: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  warnings.warn(


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,NEW_STATUS
0,5008804.0,1.0,1.0,1.0,0.0,427500.0,4.0,1.0,0.0,4.0,...,-4542.0,1.0,1.0,0.0,0.0,7.0,2.0,0,X,0
1,5008805.0,1.0,1.0,1.0,0.0,427500.0,4.0,1.0,0.0,4.0,...,-4542.0,1.0,1.0,0.0,0.0,7.0,2.0,0,X,0
2,5008806.0,1.0,1.0,1.0,0.0,112500.0,4.0,4.0,1.0,1.0,...,-1134.0,1.0,0.0,0.0,0.0,16.0,2.0,0,X,0
3,5008808.0,0.0,0.0,1.0,0.0,270000.0,0.0,4.0,3.0,1.0,...,-3051.0,1.0,0.0,1.0,1.0,14.0,1.0,0,X,0
4,5008809.0,0.0,0.0,1.0,0.0,270000.0,0.0,4.0,3.0,1.0,...,-3051.0,1.0,0.0,1.0,1.0,14.0,1.0,-22,X,0


In [36]:
df.loc[df["NEW_STATUS"]==1]

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,NEW_STATUS
732,5009628.0,0.0,0.0,0.0,0.0,238500.0,4.0,4.0,1.0,1.0,...,-3296.0,1.0,1.0,0.0,0.0,8.0,2.0,0,C,1
838,5009746.0,0.0,1.0,0.0,0.0,315000.0,0.0,1.0,1.0,1.0,...,-586.0,1.0,1.0,1.0,0.0,7.0,2.0,0,C,1
1106,5010061.0,0.0,0.0,1.0,0.0,112500.0,4.0,4.0,1.0,1.0,...,-4574.0,1.0,0.0,0.0,0.0,8.0,2.0,0,3,1
1825,5010864.0,1.0,1.0,1.0,1.0,450000.0,0.0,4.0,1.0,1.0,...,-678.0,1.0,0.0,1.0,1.0,3.0,3.0,0,C,1
2045,5018498.0,0.0,1.0,1.0,0.0,90000.0,4.0,4.0,1.0,1.0,...,-1002.0,1.0,1.0,1.0,0.0,2.0,2.0,0,C,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434798,5148934.0,1.0,1.0,0.0,0.0,108000.0,4.0,4.0,1.0,3.0,...,-1081.0,1.0,0.0,0.0,0.0,16.0,2.0,0,X,1
434805,5149190.0,1.0,1.0,0.0,1.0,450000.0,4.0,1.0,1.0,1.0,...,-502.0,1.0,0.0,1.0,1.0,3.0,3.0,0,5,1
434808,5149828.0,1.0,1.0,1.0,0.0,315000.0,4.0,4.0,1.0,1.0,...,-2420.0,1.0,0.0,0.0,0.0,10.0,2.0,0,X,1
434809,5149834.0,0.0,0.0,1.0,0.0,157500.0,0.0,1.0,1.0,1.0,...,-1325.0,1.0,0.0,1.0,1.0,11.0,2.0,0,C,1


### Training & Testing Data Preparation   

We can now slice the dataframe into X (features) and y (target), excluding ID, STATUS and NEW_STATUS.

The data will then be further split into training and testing set:

In [37]:
X = df.iloc[:,1:-3]
y = df.iloc[:,-1]

scaler = MinMaxScaler()
scaler.fit(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Recall that our target data is highly imbalanced towards 0 (over 99%). This can be solved by using Synthetic Minority Oversampling Technique (SMOTE) as follows:

In [38]:
smote = SMOTE()

X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

X_train_bal, y_train_bal = smote.fit_resample(X_train_scaled, y_train)
X_test_bal, y_test_bal = smote.fit_resample(X_test_scaled, y_test)

In [39]:
with open("objects/scaler.obj", "wb") as obj_handler:
    pickle.dump(scaler, obj_handler)

We can verify that the data has been oversampled by SMOTE:

In [40]:
y_train_bal.value_counts()

0    25307
1    25307
Name: NEW_STATUS, dtype: int64

In [41]:
y_test_bal.value_counts()

0    10848
1    10848
Name: NEW_STATUS, dtype: int64

### Export Dataset to CSV

In [42]:
X_train_bal.to_csv("processed_data/X_train_bal.csv", index=False)
X_test_bal.to_csv("processed_data/X_test_bal.csv", index=False)

y_train_bal.to_csv("processed_data/y_train_bal.csv", index=False)
y_test_bal.to_csv("processed_data/y_test_bal.csv", index=False)